## SICP 习题 （2.73）解题总结：数据导向的程序设计

SICP 习题 2.73开始了一个新的内容，关于数据导向的程序设计，这是一个很重要的设计理念，也是元编程的一个基础。

习题 2.73 的上文讲述了数据导向的程序设计思想，书中以 极坐标 和 直角坐标 为例讲了数据封装的概念，然后引入了 数据导向的程序设计理念。

题目要求我们修改之前的求导函数，变成下面这样：

In [1]:
(define (deriv exp var)
  (cond ((number? exp) 0)
	((variable? exp)
	 (if (same-variable? exp var) 1 0))
	(else ((get 'deriv (operator exp)) (operands exp)
	       var))))

这里的基本思路就是把具体的求导代码当作一种数据保存在一张表里，用操作符当作key。

当我们需要调用某个操作符的求导代码时，用该操作符作为key去表里查找，找到以后直接调用。这里又充分体现了Lisp语言里过程就是数据，数据也是过程的理念。

为了实现以上想法，我们先要设计一个简单的表插入和表查询的功能，这个也是这组题目的关键实现：

In [2]:

(define (operator exp) (car exp))

(define (operands exp) (cdr exp))

(define (assoc key records)
  (cond ((null? records) #f)
	((equal? key (caar records)) (car records))
	(else (assoc key (cdr records)))))

(define (make-table)
  (let ((local-table (list '*table*)))
    (define (lookup key-1 key-2)
      (let ((subtable (assoc key-1 (cdr local-table))))
	(if subtable
	    (let ((record (assoc key-2 (cdr subtable))))
	      (if record
		  (cdr record)
		  #f))
	    #f)))
    (define (insert! key-1 key-2 value)
      (let ((subtable (assoc key-1 (cdr local-table))))
	(if subtable
	    (let ((record (assoc key-2 (cdr subtable))))
	      (if record
		  (set-cdr! record value)
		  (set-cdr! subtable
			    (cons (cons key-2 value)
				  (cdr subtable)))))
	    (set-cdr! local-table
		      (cons (list key-1
				  (cons key-2 value))
			    (cdr local-table)))))
      'ok)
    (define (dispatch m)
      (cond ((eq? m 'lookup-proc) lookup)
	    ((eq? m 'insert-proc!) insert!)
	    (else (error "Unknown operation -- TABLE" m))))
    dispatch))

注意上面的(make-table)代码返回的是dispatch函数，该函数接受一个参数表示要对表执行的操作，如果参数是lookup-proc就是要查找表，这时再返回lookup函数，如果参数是insert-proc!就表示要插入数据，这时再返回insert!函数。

这里已经使用了消息传递的概念，希望调用某个函数时给管理对象发送一个消息，让管理对象根据消息内容返回不同的函数供调用。

对应我们在面向对象的环境，相当于是调用一个对象的方法。

然后就是给这张表插入不同求导规则的操作符和对应的代码：

In [3]:
(define operation-table (make-table))

(define get (operation-table 'lookup-proc))

(define put (operation-table 'insert-proc!))

(define (install-operations)
  (put 'deriv '+ (lambda (operands var) (make-sum (deriv (car operands) var) (deriv (cadr operands) var))))
  (put 'deriv '* (lambda (operands var)
		   (make-sum
		    (make-product  (car operands) (deriv (cadr operands) var))
		    (make-product (deriv (car operands) var) (cadr operands) ))))

  (put 'deriv '** (lambda (operands var)
		    (let ((n (cadr operands))
			  (u (car operands)))
		      (display n) (newline)
		      (display u) (newline)
		      (make-product
		       n
		       (make-product
			(make-exponentiation
			 u
			 (- n 1))
			(deriv u var))))))


)


然后把之前的各种相关函数拷贝进来：

In [4]:
(define (variable? x) (symbol? x))

(define (same-variable? v1 v2)
  (and (variable? v1) (variable? v2) (eq? v1 v2)))

(define (make-sum a1 a2)
  (define (iter sum1 sum2)
    '())
  (cond ((=number? a1 0) a2)
	((=number? a2 0) a1)
	((and (number? a1) (number? a2)) (+ a1 a2))
	(else 
	 (list '+ a1 a2))))

(define (=number? exp num)
  (and (number? exp) (= exp num)))

(define (make-product m1 m2)
  (cond ((or (=number? m1 0) (=number? m2 0)) 0)
	((=number? m1 1) m2)
	((=number? m2 1) m1)
	((and (number? m1) (number? m2)) (* m1 m2))
	(else (list '* m1 m2))))

(define (sum? x)
  (and (pair? x) (eq? (car x) '+)))

(define (addend s) (cadr s))

(define (augend s) 
   (if (null? (cdddr s)) 
       (caddr s) 
       (cons '+ (cddr s)))) 

(define (product? x)
  (and (pair? x) (eq? (car x) '*)))

(define (multiplier p) (cadr p))


(define (multiplicand p) 
   (if (null? (cdddr p)) 
       (caddr p) 
       (cons '* (cddr p)))) 


(define (make-exponentiation base exponent)
  (cond ((= exponent 0)
	 1)
	((= exponent 1)
	 base)
	(else
	 (list '** base exponent))))

(define (exponentiation? x)
  (and (pair? x)
       (eq? (car x) '**)))

(define (base exp)
  (cadr exp))

(define (exponent exp)
  (caddr exp))



再封装一个测试函数，测试函数先调用(install-operations)构建操作符和对应求导程序的关系表，供后续查找。

然后就是调用(deriv)函数，虽然在这里看起来和之前题目的调用方式完全一样，但是这个deriv的实现是完全不同的。

In [7]:


(define (start-test-2-73)

  (install-operations)
  
  (display (deriv '(+ x 3) 'x)) (newline)

  (display (deriv '(* x y) 'x)) (newline)

  (display (deriv '(* (* x y) (+ x 3)) 'x)) (newline)

  (display (deriv '(** x 6) 'x)) (newline)

  (display "if we want to call the deriv with (get (operator exp) 'deriv)") (newline)
  (display "we only need to put the actions with (put (lambda (...) ...) 'deriv)") (newline)

  (newline)) 


In [8]:
(start-test-2-73)

1
y
(+ (* x y) (* y (+ x 3)))
6
x
(* 6 (** x 5))
if we want to call the deriv with (get (operator exp) 'deriv)
we only need to put the actions with (put (lambda (...) ...) 'deriv)



然后到这里才开始看题目的具体问题。

第一小问问我们为什这种方式不能把 (number?) (same-variable)这样的函数也当做普通的操作符处理。

要回答这个问题你可以尝试真的去实现它。为了实现它你需要看这类表达式里操作符是什么，这里就傻眼了，(number?)的操作符会是所有数字，准确讲应该是所有变量加数字。因为我们需要判断(number? 1) (number? 2) (number? 3) ... 还有(number? 2.1) (number? a)...

对这种情况我们是无法现实成有限操作符的查找操作的。

第二问要求我们安装 和式 和 乘式 的求导法则，这个在上面的代码里为了测试已经加进去了。

第三问要求我们安装的 幂计算 的求导法则，这个在上面的代码里也囊括了。

第四问有点奇怪，题目问我们能不能实现(get '+ 'deriv)这样的操作，题目原来的实现是这样的(get 'deriv '+)。

题目要求新方法就是参数顺序不同了，如果希望做到这点，在(put 'deriv '+)时换个顺序就好了，用(put '+ 'deriv)